# Food Recommendation System for Diabetic Patients

#### MADS Capstone Project
##### Claire Bentzen, Tara Dehdari, Logan Van Dine

* Project description/intro

### Imports

In [59]:
import requests
import pandas as pd

### Data Extraction

#### NutritionX API

In [75]:
# API ID and key
app_key = "37d8dfed8a3b918543105661e60d25aa"
app_id = "4449c303"

# Endpoints
restaurant_url = "https://trackapi.nutritionix.com/v2/search/instant"
nutrients_url = "https://trackapi.nutritionix.com/v2/natural/nutrients"

# API Headers
headers = {
    "x-app-id": app_id,
    "x-app-key": app_key,
    "Content-Type": "application/json"
}

# List of restaurants
restaurants = ["McDonald's", "Burger King", "Taco Bell", "Chick-fil-A", "Wendy's"]

# Initialize an empty list to store data
all_menu_items = []

# Loop through each restaurant and search for menu items
for restaurant in restaurants:
    print(f"Searching for menu items from {restaurant}...")

    params = {
        "query": restaurant
    }
    
    response = requests.get(restaurant_url, headers=headers, params=params)
    
    if response.status_code == 200:
        search_data = response.json()
        menu_items = search_data.get('branded', [])

        # Loop through the menu items to gather nutritional information
        for item in menu_items:
            food_name = item.get('food_name', 'N/A')
            menu_item_data = {
                'restaurant_name': item.get('brand_name', 'N/A'),
                'food_name': food_name,
                'serving_size': item.get('serving_qty', 'N/A'),
                'serving_unit': item.get('serving_unit', 'N/A'),
                'calories': item.get('nf_calories', 'N/A'),
                'carbohydrates': item.get('nf_total_carbohydrate', 'N/A'),
                'sugars': item.get('nf_sugars', 'N/A'),
                'fats': item.get('nf_total_fat', 'N/A'),
                'saturated_fats': item.get('nf_saturated_fat', 'N/A'),
                'cholesterol': item.get('nf_cholesterol', 'N/A'),
                'sodium': item.get('nf_sodium', 'N/A'),
                'fiber': item.get('nf_dietary_fiber', 'N/A'),
                'potassium': item.get('nf_potassium', 'N/A'),
                'proteins': item.get('nf_protein', 'N/A')
            }
            
            # Check for missing nutritional info and try to get it from common foods
            if any(v == 'N/A' for v in [menu_item_data['calories'], menu_item_data['carbohydrates'], 
                                         menu_item_data['sugars'], menu_item_data['fats'],
                                         menu_item_data['saturated_fats'], menu_item_data['cholesterol'],
                                         menu_item_data['sodium'], menu_item_data['fiber'], 
                                         menu_item_data['potassium'], menu_item_data['proteins']]):
                
                # Make a search for the common food name
                common_params = {
                    "query": food_name
                }
                common_response = requests.get(restaurant_url, headers=headers, params=common_params)
                
                if common_response.status_code == 200:
                    common_data = common_response.json()
                    common_foods = common_data.get('common', [])
                    
                    if common_foods:
                        # Use the first common food to get the nutritional info
                        common_food_name = common_foods[0].get('food_name')
                        nutrients_data = {
                            "query": common_food_name
                        }
                        
                        # Make the POST request to the /natural/nutrients endpoint
                        nutrients_response = requests.post(nutrients_url, headers=headers, json=nutrients_data)
                        
                        if nutrients_response.status_code == 200:
                            nutrients_info = nutrients_response.json()
                            food_nutrients = nutrients_info.get('foods', [])
                            
                            if food_nutrients:
                                # Update menu item data with nutrients
                                food_nutrient_info = food_nutrients[0]
                                menu_item_data.update({
                                    'calories': food_nutrient_info.get('nf_calories', 'N/A'),
                                    'carbohydrates': food_nutrient_info.get('nf_total_carbohydrate', 'N/A'),
                                    'sugars': food_nutrient_info.get('nf_sugars', 'N/A'),
                                    'fats': food_nutrient_info.get('nf_total_fat', 'N/A'),
                                    'saturated_fats': food_nutrient_info.get('nf_saturated_fat', 'N/A'),
                                    'cholesterol': food_nutrient_info.get('nf_cholesterol', 'N/A'),
                                    'sodium': food_nutrient_info.get('nf_sodium', 'N/A'),
                                    'fiber': food_nutrient_info.get('nf_dietary_fiber', 'N/A'),
                                    'potassium': food_nutrient_info.get('nf_potassium', 'N/A'),
                                    'protein': food_nutrient_info.get('nf_protein', 'N/A')
                                })

            all_menu_items.append(menu_item_data)
    else:
        print(f"Error: Unable to search for {restaurant}. Status code: {response.status_code}")

# Convert the list of all menu items into a DataFrame
menu_df = pd.DataFrame(all_menu_items)

# Display the menu_info dataframe
print(menu_df)

Searching for menu items from McDonald's...
Searching for menu items from Burger King...
Searching for menu items from Taco Bell...
Searching for menu items from Chick-fil-A...
Searching for menu items from Wendy's...
      restaurant_name                                          food_name  \
0   McDonald's Canada  Egg BLT McMuffin with Shredded Lettuce (McDona...   
1          McDonald's                                       Cheeseburger   
2          McDonald's                                          Hamburger   
3          McDonald's                                              Honey   
4          McDonald's                                           Hotcakes   
..                ...                                                ...   
95            Wendy's                                  Pretzel Baconator   
96            Wendy's                                    Sausage Biscuit   
97            Wendy's                                      Sprite, Large   
98            Wendy's 

In [77]:
# Display menu_df head
menu_df.head()

,restaurant_name,food_name,serving_size,serving_unit,calories,carbohydrates,sugars,fats,saturated_fats,cholesterol,sodium,fiber,potassium,proteins
0,McDonald's Canada,Egg BLT McMuffin with Shredded Lettuce (McDona...,1,Serving,7.99,1.55,0.56,0.14,0.02,0,3.76,0.99,116.09,0.58
1,McDonald's,Cheeseburger,1,Serving,535.31,39.24,7.16,28.66,14,95.52,1176.09,2.39,443.77,30.27
2,McDonald's,Hamburger,1,Serving,540.14,40.27,None,26.56,10.52,122.04,791,None,569.52,34.28
3,McDonald's,Honey,1,Serving,63.84,17.3,17.25,0,0,0,0.84,0.04,10.92,0.06
4,McDonald's,Hotcakes,1,Serving,90.80,11.32,None,3.88,0.85,23.6,175.6,None,52.8,2.56


In [79]:
# Save the DataFrame as a CSV file
data_dir = './data/'
menu_df.to_csv(data_dir + 'menu_df.csv', index=False)

#### Dietary Recommendations

#### Patient Health Information